# CrewAI Workflows Fundamentals

## Overview
This notebook covers the core concepts and basic implementations of CrewAI Workflows. You'll learn how to create, configure, and execute multi-agent workflows for complex task automation.

## Learning Objectives
- Understand CrewAI Workflows architecture
- Create and configure agents with specific roles
- Design task workflows and dependencies
- Implement basic crew coordination patterns
- Handle workflow execution and results

## 1. Setup and Installation

First, let's install CrewAI and set up our environment.

In [ ]:
# Install CrewAI and dependencies
# !pip install crewai crewai-tools python-dotenv

# Import required libraries
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool, FileReadTool, DirectoryReadTool

# Load environment variables
load_dotenv()

# Set up API keys (you'll need to add these to your .env file)
os.environ["OPENAI_API_KEY"] = os.getenv("OPEN_ROUTER_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
os.environ['LITELLM_LOG'] = 'DEBUG' 
os.environ['OPENAI_API_BASE'] = 'https://openrouter.ai/api/v1'
os.environ['OPENAI_BASE_URL'] = 'https://openrouter.ai/api/v1'


print("CrewAI setup complete!")

## 2. Core Concepts

### 2.1 Agents
Agents are the core building blocks of CrewAI. Each agent has a specific role, goal, and backstory that defines their behavior and expertise.

In [ ]:
llm = LLM(
        model='openai/gpt-4o',
        api_key=os.getenv('OPEN_ROUTER_KEY'),
        base_url="https://openrouter.ai/api/v1"
    )

# Example: Creating a Research Agent
research_agent = Agent(
    role='Senior Research Analyst',
    goal='Uncover cutting-edge developments in AI and data science',
    backstory="""You work at a leading tech think tank.
    Your expertise lies in identifying emerging trends.
    You have a knack for dissecting complex data and presenting actionable insights.""",
    verbose=True,
    allow_delegation=False,
    tools=[SerperDevTool()],
    llm=llm,
)

print(f"Agent created: {research_agent.role}")
print(f"Goal: {research_agent.goal}")

### 2.2 Tasks
Tasks define what needs to be accomplished. They specify the description, expected output, and which agent should handle them.

In [ ]:
# Example: Creating a Research Task
research_task = Task(
    description="""Conduct a comprehensive analysis of the latest advancements in GPT-4 and LLM applications.
    Identify key trends, breakthrough technologies, and potential industry impacts.
    Your final answer MUST be a full analysis report""",
    expected_output="A comprehensive 3 paragraphs long report on the latest AI developments.",
    agent=research_agent
)

print(f"Task created: {research_task.description[:50]}...")
print(f"Assigned to: {research_task.agent.role}")

### 2.3 Crews
Crews coordinate multiple agents and tasks to achieve complex objectives through structured workflows.

In [ ]:
# Example: Creating a simple crew
research_crew = Crew(
    agents=[research_agent],
    tasks=[research_task],
    process=Process.sequential,
    verbose=True,
    llm=llm,
    custom_llm_provider="openrouter",
)

print(f"Crew created with {len(research_crew.agents)} agent(s) and {len(research_crew.tasks)} task(s)")
print(f"Process type: {research_crew.process}")

## 3. Basic Workflow Example

Let's create a complete workflow with multiple agents working together on a content creation pipeline.

In [ ]:
# Define multiple agents for a content creation workflow

# 1. Content Planner Agent
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="""You're working on planning a blog article about the topic: {topic}.
    You collect information that helps the audience learn something and make informed decisions.
    Your work is the basis for the Content Writer to write an article on this topic.""",
    tools=[SerperDevTool()],
    allow_delegation=False,
	verbose=True
)

# 2. Content Writer Agent
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate opinion piece about the topic: {topic}",
    backstory="""You're working on writing a new opinion piece about the topic: {topic}.
    You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic.
    You follow the main objectives and direction of the outline, as provided by the Content Planner.
    You also provide objective and impartial insights and back them up with information provide by the Content Planner.
    You acknowledge in your opinion piece when your statements are opinions as opposed to objective statements.""",
    allow_delegation=False,
    verbose=True
)

# 3. Editor Agent
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with the writing style of the organization.",
    backstory="""You are an editor who receives a blog post from the Content Writer.
    Your goal is to review the blog post to ensure that it follows journalistic best practices,
    provides balanced viewpoints when providing opinions or assertions, and also avoids major controversial topics
    or opinions when possible.""",
    allow_delegation=False,
    verbose=True
)

print("Content creation agents created successfully!")

In [ ]:
# Define tasks for the content creation workflow

# 1. Planning Task
plan_task = Task(
    description="""1. Prioritize the latest trends, key players, and noteworthy news on {topic}.
    2. Identify the target audience, considering their interests and pain points.
    3. Develop a detailed content outline including an introduction, key points, and a call to action.
    4. Include SEO keywords and relevant data or sources.""",
    expected_output="A comprehensive content plan document with an outline, audience analysis, and SEO keywords.",
    agent=planner,
)

# 2. Writing Task
write_task = Task(
    description="""1. Use the content plan to craft a compelling blog post on {topic}.
    2. Incorporate SEO keywords naturally.
    3. Sections/Subtitles are properly named in an engaging manner.
    4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
    5. Proofread for grammatical errors and alignment with the brand's voice.""",
    expected_output="A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.",
    agent=writer,
)

# 3. Editing Task
edit_task = Task(
    description="""Proofread the given blog post for grammatical errors and alignment with the brand's voice.""",
    expected_output="A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.",
    agent=editor
)

print("Content creation tasks defined successfully!")

In [ ]:
# Create and execute the content creation crew
content_crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan_task, write_task, edit_task],
    process=Process.sequential,
    verbose=True
)

print("Content creation crew assembled!")
print(f"Agents: {[agent.role for agent in content_crew.agents]}")
print(f"Tasks: {len(content_crew.tasks)}")

In [ ]:
# Execute the workflow
topic = "Artificial Intelligence in Healthcare"

print(f"Starting content creation workflow for topic: {topic}")
print("=" * 50)

# Note: This will make actual API calls and may take a few minutes
# result = content_crew.kickoff(inputs={'topic': topic})

# For demonstration purposes, we'll show the structure without executing
print("Workflow structure:")
print("1. Content Planner researches and creates outline")
print("2. Content Writer creates blog post based on outline")
print("3. Editor reviews and finalizes the content")
print("\nTo execute: result = content_crew.kickoff(inputs={'topic': topic})")

## 4. Working with Tools

CrewAI agents can use various tools to enhance their capabilities. Let's explore some common tools.

In [ ]:
# Import additional tools
from crewai_tools import (
    FileReadTool,
    DirectoryReadTool,
    SerperDevTool,
    WebsiteSearchTool,
    CodeDocsSearchTool
)

# Initialize tools
file_read_tool = FileReadTool()
directory_read_tool = DirectoryReadTool()
search_tool = SerperDevTool()
web_search_tool = WebsiteSearchTool()

print("Tools initialized:")
print(f"- File Read Tool: {type(file_read_tool).__name__}")
print(f"- Directory Read Tool: {type(directory_read_tool).__name__}")
print(f"- Search Tool: {type(search_tool).__name__}")
print(f"- Website Search Tool: {type(web_search_tool).__name__}")

In [ ]:
# Create an agent with multiple tools
data_analyst = Agent(
    role="Data Analyst",
    goal="Analyze data files and provide insights",
    backstory="""You are an expert data analyst with deep knowledge of data processing,
    statistical analysis, and data visualization. You can read various file formats
    and extract meaningful insights from complex datasets.""",
    tools=[file_read_tool, directory_read_tool, search_tool],
    verbose=True
)

# Create a task for data analysis
analysis_task = Task(
    description="""Analyze the current directory structure and identify any data files.
    If data files are found, provide a summary of their contents and potential insights.
    If no data files are found, suggest what types of data analysis could be performed.""",
    expected_output="A comprehensive report on available data files and analysis recommendations.",
    agent=data_analyst
)

print("Data analyst agent created with tools")
print(f"Available tools: {[tool.__class__.__name__ for tool in data_analyst.tools]}")

## 5. Process Types and Coordination

CrewAI supports different process types for task execution coordination.

In [ ]:
# Sequential Process Example
sequential_crew = Crew(
    agents=[planner, writer],
    tasks=[plan_task, write_task],
    process=Process.sequential,
    verbose=1
)

print("Sequential Process:")
print("- Tasks execute one after another")
print("- Each task waits for the previous one to complete")
print("- Output from one task can be input to the next")
print(f"Task order: {[task.agent.role for task in sequential_crew.tasks]}")

In [ ]:
# Hierarchical Process Example (requires manager agent)
manager = Agent(
    role="Project Manager",
    goal="Coordinate team efforts and ensure quality deliverables",
    backstory="""You are an experienced project manager who excels at coordinating teams,
    delegating tasks, and ensuring project objectives are met on time and within scope.""",
    allow_delegation=True,
    verbose=True
)

hierarchical_crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan_task, write_task, edit_task],
    process=Process.hierarchical,
    manager_agent=manager,
    verbose=1
)

print("Hierarchical Process:")
print("- Manager agent coordinates and delegates tasks")
print("- Allows for dynamic task assignment")
print("- Manager reviews and guides the workflow")
print(f"Manager: {hierarchical_crew.manager_agent.role}")

## 6. Error Handling and Validation

Proper error handling ensures robust workflow execution.

In [ ]:
def safe_crew_execution(crew, inputs=None):
    """
    Safely execute a crew with proper error handling
    """
    try:
        print("Starting crew execution...")
        
        # Validate crew configuration
        if not crew.agents:
            raise ValueError("Crew must have at least one agent")
        
        if not crew.tasks:
            raise ValueError("Crew must have at least one task")
        
        # Check if all tasks have assigned agents
        for i, task in enumerate(crew.tasks):
            if not task.agent:
                raise ValueError(f"Task {i} does not have an assigned agent")
        
        print("✓ Crew validation passed")
        
        # Execute crew (commented out to avoid API calls in demo)
        result = crew.kickoff(inputs=inputs)
        return result
        
        print("✓ Crew execution would start here")
        return "Demo execution completed successfully"
        
    except ValueError as ve:
        print(f"❌ Validation Error: {ve}")
        return None
    except Exception as e:
        print(f"❌ Execution Error: {e}")
        return None

# Test the safe execution function
result = safe_crew_execution(content_crew, {'topic': 'AI Safety'})
print(f"Result: {result}")

## 7. Configuration and Customization

Learn how to customize agent behavior and crew settings.

In [ ]:
# Advanced agent configuration
advanced_agent = Agent(
    role="Senior AI Researcher",
    goal="Conduct thorough research on AI topics",
    backstory="Expert in AI research with 10+ years experience",
    verbose=True,
    allow_delegation=True,
    max_iter=5,  # Maximum iterations for task completion
    tools=[search_tool]
)

# Advanced task configuration
advanced_task = Task(
    description="Research the latest developments in large language models",
    expected_output="A detailed research report with citations",
    agent=advanced_agent,
    async_execution=False,  # Synchronous execution
    output_file="research_report.md"  # Save output to file
)

print("Advanced Configuration:")
print(f"Agent max iterations: {advanced_agent.max_iter}")
print(f"Task output file: {advanced_task.output_file}")
print(f"Task async execution: {advanced_task.async_execution}")

## 8. Best Practices and Tips

Key recommendations for effective CrewAI workflow design.

In [ ]:
# Best Practices Demonstration

def create_effective_agent(role, goal, backstory, tools=None):
    """
    Create an agent following best practices
    """
    return Agent(
        role=role,
        goal=goal,
        backstory=backstory,
        tools=tools or [],
        verbose=True,
        allow_delegation=False,  # Start with False, enable as needed
        memory=True  # Enable memory for better context retention
    )

def create_clear_task(description, expected_output, agent):
    """
    Create a task with clear, specific instructions
    """
    return Task(
        description=description,
        expected_output=expected_output,
        agent=agent
    )

# Example of well-designed workflow
qa_specialist = create_effective_agent(
    role="Quality Assurance Specialist",
    goal="Ensure all outputs meet high quality standards",
    backstory="You are a meticulous QA specialist with expertise in content review and quality control.",
    tools=[file_read_tool]
)

qa_task = create_clear_task(
    description="Review the final content for accuracy, clarity, and completeness. Check for any errors or inconsistencies.",
    expected_output="A quality assessment report with recommendations for improvement if needed.",
    agent=qa_specialist
)

print("Best Practices Applied:")
print("✓ Clear, specific role definitions")
print("✓ Well-defined goals and backstories")
print("✓ Appropriate tool selection")
print("✓ Specific task descriptions and expected outputs")
print("✓ Memory enabled for context retention")

## 9. Exercise: Build Your Own Workflow

Now it's your turn! Create a simple workflow for a use case of your choice.

In [ ]:
# Exercise: Create a customer support workflow
# TODO: Implement the following:
# 1. Ticket Classifier Agent - categorizes support tickets
# 2. Support Specialist Agent - provides solutions
# 3. Follow-up Agent - creates follow-up communications

# Your code here...

# Example structure:
# classifier = Agent(...)
# specialist = Agent(...)
# follow_up = Agent(...)

# classify_task = Task(...)
# solve_task = Task(...)
# follow_up_task = Task(...)

# support_crew = Crew(...)

print("Exercise: Implement your customer support workflow above!")
print("Consider:")
print("- What tools might each agent need?")
print("- How should tasks be sequenced?")
print("- What should the expected outputs be?")

## 10. Summary and Next Steps

### Key Concepts Covered
- **Agents**: Autonomous entities with specific roles and capabilities
- **Tasks**: Specific objectives with clear descriptions and expected outputs
- **Crews**: Coordination mechanisms for multi-agent workflows
- **Tools**: Extensions that enhance agent capabilities
- **Processes**: Different execution patterns (sequential, hierarchical)

### Best Practices Learned
- Define clear, specific roles and goals for agents
- Provide detailed task descriptions and expected outputs
- Choose appropriate tools for each agent's needs
- Implement proper error handling and validation
- Test workflows incrementally

### Next Steps
1. Practice creating workflows for different domains
2. Experiment with different agent configurations
3. Explore the CrewAI Workflows Advanced notebook
4. Build production-ready workflows with monitoring and logging

### Resources
- [CrewAI Documentation](https://docs.crewai.com/)
- [CrewAI Tools Repository](https://github.com/joaomdmoura/crewAI-tools)
- [Community Examples](https://github.com/joaomdmoura/crewAI-examples)